IA PI VII - reconhecimento de cores 

In [1]:
!pip uninstall -y tensorflow

Uninstalling tensorflow-2.5.0:
  Successfully uninstalled tensorflow-2.5.0


In [2]:
!pip install tensorflow-gpu==2.0

     |████████████████████████████████| 380.8MB 42kB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 3.8MB 33.4MB/s 
     |████████████████████████████████| 450kB 43.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=c46cc18439edae481297ec1f844ea51c1d9b2bd997539c0d6dd2df8f156f2108
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tens

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import numpy as np
import os
import tensorflow as tf

In [4]:
dataset_path = '/content/drive/MyDrive/datasets/PI-VII/dataset'

In [5]:
train_dir = os.path.join(dataset_path,'train')
validation_dir = os.path.join(dataset_path,'validation')

In [6]:
data_gen_train = ImageDataGenerator(rescale=1./255)
data_gen_validation = ImageDataGenerator(rescale=1./255)

In [7]:
img_width = 128
img_height = 128

In [8]:
train_generator = data_gen_train.flow_from_directory(train_dir,target_size=(img_width,img_height), batch_size=32,color_mode='rgb',seed=42,class_mode='categorical')
validation_generator = data_gen_validation.flow_from_directory(validation_dir,target_size=(img_height,img_width), batch_size=32,color_mode='rgb',seed=42,class_mode='categorical')

Found 1085 images belonging to 4 classes.
Found 346 images belonging to 4 classes.


In [9]:
distribute = tf.distribute.MirroredStrategy()

In [10]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=(img_height,img_width, 3)))

model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding="same", activation="relu"))

model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding="same", activation="relu"))

model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding="same", activation="relu"))

model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

model.add(tf.keras.layers.Conv2D(filters=1028, kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.Conv2D(filters=1028, kernel_size=3, padding="same", activation="relu"))

model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

model.add(tf.keras.layers.Flatten())

# model.add(tf.keras.layers.Dropout(0.1)
# model.add(tf.keras.layers.Dense(units=1028, activation='relu'))

# model.add(tf.keras.layers.Dropout(0.1)
# model.add(tf.keras.layers.Dense(units=512, activation='relu'))

# model.add(tf.keras.layers.Dropout(0.1)
# model.add(tf.keras.layers.Dense(units=256, activation='relu'))

# model.add(tf.keras.layers.Dropout(0.1)
model.add(tf.keras.layers.Dense(units=4, activation='softmax'))
model.compile(loss="categorical_crossentropy",
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=["accuracy"])
# model.summary()

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 128)     36992     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 256)       295168    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 256)       590080    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 256)       0

In [12]:
# history = model.fit_generator(train_generator, epochs=50, validation_data=validation_generator)
history = model.fit(train_generator, epochs=50,validation_data=validation_generator)

Train for 34 steps, validate for 11 steps
Epoch 1/50
 2/34 [>.............................] - ETA: 34:41 - loss: 1.3861 - accuracy: 0.3125

KeyboardInterrupt: ignored

In [ ]:
valid_loss, valid_accuracy = model.evaluate_generator(validation_generator)
# predict = model.predict_generator(validation_generator,validation_generator.n//validation_generator.batch_size,verbose=1)


In [ ]:
valid_accuracy

Salvar Model 

In [ ]:
model_name = "/content/drive/MyDrive/datasets/PI-VII/"+ "IA_PI-VII.h5"


In [ ]:
model.save("/content/drive/MyDrive/datasets/PI-VII/"+ "IA_PI-VII.h5")

In [ ]:
os.remove(model_name)
tf.keras.models.save_model(model,model_name)